In [1]:
import sys
sys.path.insert(1, r"Q:\IlMIoDrive\magistrale\1anno\terzo_periodo\MAS\analysis")
import json as js
import bz2
import copy
import os 
from threading import Thread
import pickle

Insert path to year folder (*bz_files_path*) (containing all months folders) and saving folder (*save_path*) here. The latter must contain a subfolder for every month , called *01*, $02$ etc...

In [2]:
bz_files_path = r"D:\TwitterUsers\archiveTeam\months\2012\\"
save_path = r"D:\TwitterUsers\archiveTeam\saved_objs_with_keywords\\"

bz2 archiveTeam parsing:

In [3]:
fldr = "01"
listOfFiles = list()
for (dirpath, dirnames, filenames) in os.walk(bz_files_path+fldr):
    listOfFiles += [os.path.join(dirpath, file) for file in filenames]

In [4]:
len(listOfFiles)

44540

Useful chunks
```python
%%time
bz_file = bz2.BZ2File(listOfFiles[2])
line_list = bz_file.readlines()
tweets = [js.loads(statusObj.decode('utf8').replace("\r\n","")) for statusObj in line_list]
indices = [i  for (i,tweet) in enumerate(tweets) if parse_tweet(tweet,["replied_to_tweet_id"]) != None and parse_tweet(tweet,["replied_to_user_id"]) == None ]
print(indices)
int_tweet = [tweet for tweet in tweets if parse_tweet(tweet,["id"]) == 153370379715481600 ]
int_tweet

objs = []
objs.extend([{"tweet_id":parse_tweet(tweet, ["id"]),"retweeted_tweet_id":parse_tweet(tweet,["retweeted_status","id"]),"retweeted_user_id":parse_tweet(tweet,["retweeted_status","user","id"]),"replied_to_tweet_id":parse_tweet(tweet, ["in_reply_to_status_id"]),"replied_to_user_id":parse_tweet(tweet, ["in_reply_to_user_id"]),"hashtags":[dct["text"] for dct in parse_tweet(tweet,["entities","hashtags"])], "tweet_created_at":parse_tweet(tweet,["created_at"]),"text":parse_tweet(tweet,["text"]),"user_mentions": [mention["id"] for mention in parse_tweet(tweet,["entities","user_mentions"])],"user_id":parse_tweet(tweet,["user","id"]),"profile_created_at":parse_tweet(tweet,["user","created_at"]),'followers_count': parse_tweet(tweet,["user","followers_count"]),'friends_count': parse_tweet(tweet,["user","friends_count"]), "statuses_count":parse_tweet(tweet, ["user","statuses_count"]), "favourites_count":parse_tweet(tweet, ["user","favourites_count"]) } for tweet in tweets if parse_tweet(tweet, ["user","lang"])=="en" ])
objs[0]
sum([1 if tweet['replied_to_tweet_id'] != None else 0 for tweet in objs])
mentions_indices = [i  for (i,tweet) in enumerate(objs) if tweet["replied_to_tweet_id"] == None and tweet["replied_to_user_id"] != None ]
objs[53]
```

In [5]:
class bz2Thread(Thread):
    
    def __init__(self,name, paths, save_path, save_every = 1440):
        Thread.__init__(self)
        self.name  = name
        self.paths = paths
        self.save_path = save_path
        self.save_every = save_every
        
        
    def parse_tweet(self,tweet,path):
        result  = copy.deepcopy(tweet)
    
        try:
            for field in path:
                result = result[field]
            return result
        except KeyError:
            return []
    
    
    def run(self):
        i = 0
        j = 0
        total = len(self.paths)
        objs = []
        for path in self.paths:
            bz_file = bz2.BZ2File(path)
            line_list = bz_file.readlines()
            tweets = [js.loads(statusObj.decode('utf8').replace("\r\n","")) for statusObj in line_list]
            objs.extend([{"tweet_id":self.parse_tweet(tweet, ["id"]),"retweeted_tweet_id":self.parse_tweet(tweet,["retweeted_status","id"]),"retweeted_user_id":self.parse_tweet(tweet,["retweeted_status","user","id"]),"hashtags":[dct["text"] for dct in self.parse_tweet(tweet,["entities","hashtags"])], "tweet_created_at":self.parse_tweet(tweet,["created_at"]),"text":self.parse_tweet(tweet,["text"]),"user_mentions": [mention["id"] for mention in self.parse_tweet(tweet,["entities","user_mentions"])],"user_id":self.parse_tweet(tweet,["user","id"]),"profile_created_at":self.parse_tweet(tweet,["user","created_at"]),'followers_count': self.parse_tweet(tweet,["user","followers_count"]),'friends_count': self.parse_tweet(tweet,["user","friends_count"]), "statuses_count":self.parse_tweet(tweet, ["user","statuses_count"]), "favourites_count":self.parse_tweet(tweet, ["user","favourites_count"]) } for tweet in tweets if self.parse_tweet(tweet, ["user","lang"])=="en" ])
            i += 1
            if i%self.save_every == 0:
                with open(self.save_path+str(j)+".txt", "wb") as f:
                    pickle.dump(objs, f)
                objs = []
                j += 1
            
            if self.name == "thread0":
                print("done",i,"out of", total,end = "\r")
        
        if len(objs) > 0:
            with open(self.save_path+str(j)+".txt", "ab") as f:
                pickle.dump(objs, f)
                objs = []
            
    

In [6]:
bz2Thread1 = bz2Thread(name = "thread0",paths = listOfFiles ,save_path = save_path + fldr +"\objs_")

In [7]:
%%time
bz2Thread1.start()
bz2Thread1.join()

Wall time: 20h 31min 28s245 out of 44540 12225 out of 44540 24949 out of 44540 44392 out of 44540
